# TP3 - DB Normalization and Querying

The objectives of this TP are:
1. Apply normalization 1NF -> 2NF -> 3NF
2. Perform SQL queries on the normalized database

In this TP, we will use a database **`wine.db`** (available in the course's website) containing wine information related to 'production' and 'sales'. 

> Production <---> Wine <---> Sales


---

### Working with db files in Jupyter
- Python provides an interface for SQLite through the *sqlite3* module
- The **`%%sql`** magic builds upon it (and other tools) to enable the usage of SQL commands within a Jupyter Notebook as in common SQL clients.
- Before proceeding, make sure that **`wine.db`** is in the same path as this notebook.
  - If **`wine.db`** is not in the same path, an empty **`wine.db`** file will be created, resulting in errors in later steps of the TP.
- The SQLite module in Python commits transactions automatically, this means that any change in the DB is immediately written to the file, e.g. creating/deleting tables.
  -  For this reason, it is recommended to have a backup of **`wine.db`** as it is provided in the course's website.

---

**`wine.db`** contains the following unnormalized tables:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

From wikipedia:

__Cru__: Often used to indicate a specifically named and legally defined vineyard or ensemble of vineyards and the vines "which grow on [such] a reputed terroir; by extension of good quality." The term is also used to refer to the wine produced from such vines.


<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


In [1]:
import sqlite3    # Python interface for SQLite databases

In [28]:
def printSchema(connection):
    # Function to print the DB schema
    # Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [29]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)           # An usefull way to viualize the content of the database

Database schema:
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
client:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
  4: REGION(TEXT)
production:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
sales1:
  0: NP(NUM)
  1: NV(NUM)
  2: QTE(NUM)
sales2:
  0: NB(NUM)
  1: NV(NUM)
  2: QTE(NUM)
  3: DATES(NUM)
  4: LIEU(TEXT)
wine1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
wine2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


From this point we will use __%%sql__ magic

In [30]:
%load_ext sql
%sql sqlite:///wine.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


# PART I: Database normalization

The first task on this TP is the normalization of the wine data. In its current state both tables **Master1** and **Master2** are in the First Normal Form (1NF).

By inspecting the content of these tables we can see that multiple tuples have NULL values.

In [31]:
%%sql SELECT NV, NP
FROM Master1;

 * sqlite:///wine.db
Done.


NV,NP
None,3
None,6
None,8
None,16
None,17
None,18
None,19
None,20
None,21
None,26


* Notice that Jupyter *displays* 'None' instead of 'NULL'. 
  - This is only to comply with python notation.
* To account for NULL values, your SQL queries must test explicitly for 'NULL'.

Another problem in **Master1** and **Master2** is data redundancy, for example:

In [32]:
%%sql SELECT *
FROM Master1
WHERE NV = 45;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
45,Chiroubles,None,1983,90,2,Boxler,Albert,Alsace
45,Chiroubles,None,1983,912,67,Descombes,Jean Ernest,Beaujolais
45,Chiroubles,None,1983,98,71,Chalandard,Danile,Jura
45,Chiroubles,None,1983,540,78,Michlel,Pierre Emile,Jura
45,Chiroubles,None,1983,450,86,Dumazet,Marc,Rhone


---

Additional resource for Normalization:

https://www.youtube.com/watch?v=UrYLYV7WSHM

---

#### Exercise 1.1

Given the set of the following FDs:

NV -> CRU, DEGRE, MILL

NP -> NOM, PRENOM, REGION

NV, NP -> QTE


Convert table **Master1** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.
  
Remember that **`wine.db`** contains information related to wine 'production' and 'sells'.

> Production <---> Wine <---> Sales


In [33]:
# Source code

def to_set(x):
  """Convert input int, string, list, tuple, set -> set"""
  if type(x) == set:
    return x
  elif type(x) in [list, set]:
    return set(x)
  elif type(x) in [str, int]:
    return set([x])
  else:
    raise Exception("Unrecognized type.")

def fd_to_str(lr_tuple):
    lhs = lr_tuple[0]
    rhs = lr_tuple[1]
    return ",".join(to_set(lhs)) + " -> " + ",".join(to_set(rhs))

def fds_to_str(fds): return "\n\t".join(map(fd_to_str, fds))

def set_to_str(x): return "{" + ",".join(x) + "}"

def fd_applies_to(fd, x): 
    lhs, rhs = map(to_set, fd)
    return lhs.issubset(x)

def print_setup(A, fds):
    print("Attributes = " + set_to_str(A))
    print("FDs = \t" + fds_to_str(fds))

def print_fds(fds):
    print("FDs = \t" + fds_to_str(fds))    
    
def compute_closure(x, fds, verbose=False):
    bChanged = True        # We will repeat until there are no changes.
    x_ret    = x.copy()    # Make a copy of the input to hold x^{+}
    while bChanged:
        bChanged = False   # Must change on each iteration
        for fd in fds:     # loop through all the FDs.
            (lhs, rhs) = map(to_set, fd) # recall: lhs -> rhs
            if fd_applies_to(fd, x_ret) and not rhs.issubset(x_ret):
                x_ret = x_ret.union(rhs)
                if verbose:
                    print("Using FD " + fd_to_str(fd))
                    print("\t Updated x to " + set_to_str(x_ret))
                bChanged = True
    return x_ret

def is_superkey_for(A, X, fds, verbose=False): 
    return X.issubset(compute_closure(A, fds, verbose=verbose))

import itertools
def findsubsets(S,m):
    return set(itertools.combinations(S, m))
def closure(X, fds, verbose=False):
    c = []
    for size in range(1, len(X)):
        subsets = findsubsets(X, size) 
        for SA in subsets:      # loop through all the subsets.
            cl = compute_closure(set(SA), fds, verbose)
            if len(cl.difference(SA)) > 0: 
                c.extend([(set(SA), cl.difference(SA))])
    return c

def superkeys(X, fds, verbose=False):
    c = []
    for size in range(1, len(X)):
        subsets = findsubsets(X, size)
        for SA in subsets:
            cl = compute_closure(set(SA), fds, verbose)
            if cl == X and len(cl.difference(SA)) > 0: ## cl = X
                c.extend([SA])
    return c

def keys(att, fds):
    cles=[]
    for i in range(1, len(att)):
        subsets = findsubsets(att, i)
        for ss in subsets:
            if is_key_for(set(ss), att, fds):
                cles.append(ss)
    return cles

def is_key_for(A, X, fds):
    subsets = set(itertools.combinations(A, len(A)-1))
    return is_superkey_for(A, X, fds) and \
        all([not is_superkey_for(set(SA), X, fds) for SA in subsets])

In [34]:
attributes  = set(["nv", "cru", "degre", "mill", "qte", "np", "nom", "prenom", "region"]) # These are the attribute set.
fds = [(set(["nv"]),["cru", "degre", "mill"]),
         (set(["np"]), ["nom", "prenom", "region"]),
         (set(["nv", "np"]), "qte")]

print_setup(attributes, fds)

Attributes = {region,nv,qte,mill,np,degre,nom,cru,prenom}
FDs = 	nv -> degre,cru,mill
	np -> region,prenom,nom
	nv,np -> qte


In [35]:
k = keys(attributes, fds)
print("these are the keys of the Master 1 table", k)

these are the keys of the Master 1 table [('nv', 'np')]


#### Exercise 1.2

Given the set of the following FDs:

NV -> CRU, DEGRE, MILL

NB -> NOM, PRENOM, TYPE

NV, NB -> DATES, LIEU, QTE

LIEU -> REGION

Convert table **Master2** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.

**Note:** For this part, consider that a wine can be bought in multiple locations and multiple times per day.

In [36]:
attributes  = set(["nv", "cru", "degre", "mill", "qte", "nb", "nom", "prenom", "region", "lieu", "dates", "type"]) # These are the attribute set.
fds = [(set(["nv"]),["cru", "degre", "mill"]),
         (set(["nb"]), ["nom", "prenom", "type"]),
         (set(["nv", "nb"]), ["dates", "lieu", "qte"]),
         (set(["lieu"]),["region"])]

print_setup(attributes, fds)

Attributes = {region,nv,qte,mill,nb,degre,dates,nom,lieu,type,cru,prenom}
FDs = 	nv -> degre,cru,mill
	nb -> type,prenom,nom
	nv,nb -> dates,lieu,qte
	lieu -> region


In [37]:
k = keys(attributes, fds)
print("these are the keys of the Master 2 table", k)

these are the keys of the Master 2 table [('nv', 'nb')]


[Write your answer here]

The relation between the proposed Wine and the Client Tables is of Many to Many (because a wine can be bought by 1 or more clients and a client can buy one or more wines) therefore the suggestion is to create those two tables each with their keys and having a third one having both keys.

Provided the NB and NV columns are the keys for the entire Master 2 table,  I could create a three tables: Client, Wine and Sales with the following columns:

For Wine:
    NV (Key), CRU, DEGRE, MILL
    
For Clients:
    NB(Key), NOM, PRENOM, TYPE, REGION
    
For Sales:
     NV(Foreign Key of the Wine table),  NB (Foreign Key of the Client table), QTE, DATES, LIEU
     
QTE, DATES and LIEU are dependant on both the Wine and the Client table (due to the fact that nb,nv ->qte,dates,lieu)

Once you have defined the 2NF or 3NF (as applicable) we need to split the data into new tables.

A table can be created from the result of a query.

In the following example we will create a new table "dummy" to store the different values of alcohol content.

In [38]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1;

 * sqlite:///wine.db
Done.
Done.


[]

In [39]:
print("\nContent of the database")
printSchema(conn)


Content of the database
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
client:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
  4: REGION(TEXT)
dummy:
  0: DEGRE(NUM)
production:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
sales1:
  0: NP(NUM)
  1: NV(NUM)
  2: QTE(NUM)
sales2:
  0: NB(NUM)
  1: NV(NUM)
  2: QTE(NUM)
  3: DATES(NUM)
  4: LIEU(TEXT)
wine1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
wine2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


In [59]:
%%sql
SELECT *
FROM dummy;

 * sqlite:///wine.db
Done.


DEGRE
None
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8


Looking into "dummy", we notice that our query includes NULL. This is not allowed if we were to use DEGRE as key for a table.

To correct this, we need to change the query to explicitly test for NULL as follows:

In [40]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1
WHERE DEGRE IS NOT NULL;

SELECT *
FROM dummy;

 * sqlite:///wine.db
Done.
Done.
Done.


DEGRE
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8
10.7


Notice that we use `NULL` given that `None` is only used for display.

In [41]:
# Remove "dummy" table
%sql DROP TABLE IF EXISTS dummy;

 * sqlite:///wine.db
Done.


[]

#### Exercise 1.3

Create the new tables from Master1:

In [42]:
%%sql DROP TABLE IF EXISTS wine1;

-- Create wine1 table

CREATE TABLE wine1 AS
SELECT DISTINCT NV, CRU, DEGRE, MILL
FROM MASTER1
WHERE NV IS NOT NULL;

SELECT *
FROM wine1;

 * sqlite:///wine.db
Done.
Done.
Done.


NV,CRU,DEGRE,MILL
1,Mercurey,11.5,1980
2,Julienas,11.3,1974
3,Savigny les Beaunes,12.1,1978
4,Mercurey,10.9,1980
5,Pommard,11.7,1976
6,Mercurey,11.2,1981
7,Grands Echezeaux,11.7,1968
8,Cotes de Beaune Villages,12.3,1975
9,Chapelle Chambertin,11.9,1973
10,Beaujolais Villages,11.8,1979


In [43]:
%%sql DROP TABLE IF EXISTS production;

-- Create production table

CREATE TABLE production AS
SELECT DISTINCT NP, NOM, PRENOM, REGION
FROM MASTER1
WHERE NP IS NOT NULL;

SELECT * FROM production;

 * sqlite:///wine.db
Done.
Done.
Done.


NP,NOM,PRENOM,REGION
3,Six,Paul,Alsace
6,Marmagne,Bernard,Bourgogne
8,Lioger d'Harduy,Gabriel,Bourgogne
16,Barbin,Bernard,Bourgogne
17,Faiveley,Guy,Bourgogne
18,Tramier,Jean,Bourgogne
19,Dupaquier,Roger,Bourgogne
20,Lamy,Jean,Bourgogne
21,Cornu,Edmond,Bourgogne
26,Violot,Gilbert,Bourgogne


In [46]:
%%sql DROP TABLE IF EXISTS sales1;

-- Create sales1 table

CREATE TABLE sales1 AS
SELECT NP, NV, QTE
FROM MASTER1;

SELECT * FROM sales1;

 * sqlite:///wine.db
Done.
Done.
Done.


NP,NV,QTE
3,None,None
6,None,None
8,None,None
16,None,None
17,None,None
18,None,None
19,None,None
20,None,None
21,None,None
26,None,None


#### Exercise 1.4

Create the new tables from Master2:

In [47]:
%%sql DROP TABLE IF EXISTS wine2;

-- Create wine2 table

CREATE TABLE wine2 AS
SELECT DISTINCT NV, CRU, DEGRE, MILL
FROM MASTER2
WHERE NV IS NOT NULL;

SELECT *
FROM wine2;

 * sqlite:///wine.db
Done.
Done.
Done.


NV,CRU,DEGRE,MILL
1,Mercurey,11.5,1980
2,Julienas,11.3,1974
3,Savigny les Beaunes,12.1,1978
4,Mercurey,10.9,1980
5,Pommard,11.7,1976
6,Mercurey,11.2,1981
7,Grands Echezeaux,11.7,1968
8,Cotes de Beaune Villages,12.3,1975
9,Chapelle Chambertin,11.9,1973
10,Beaujolais Villages,11.8,1979


In [48]:
%%sql DROP TABLE IF EXISTS client;

-- Create client table

CREATE TABLE client AS
SELECT DISTINCT NB, NOM, PRENOM, TYPE, REGION
FROM MASTER2
WHERE NB IS NOT NULL;

SELECT * FROM client ORDER BY NB;

 * sqlite:///wine.db
Done.
Done.
Done.


NB,NOM,PRENOM,TYPE,REGION
1,Aristote,None,petit,ÎLE-DE-FRANCE
2,Artaud,Antonin,moyen,NOUVELLE-AQUITAINE
2,Artaud,Antonin,moyen,BOURGOGNE-FRANCHE-COMTÉ
3,Aron,Raymond,gros,GRAND EST
3,Aron,Raymond,gros,NOUVELLE-AQUITAINE
3,Aron,Raymond,gros,ÎLE-DE-FRANCE
4,Apollinaire,Guillaume,moyen,ÎLE-DE-FRANCE
5,Audiberti,Jacques,petit,AUVERGNE-RHÔNE-ALPES
5,Audiberti,Jacques,petit,ÎLE-DE-FRANCE
5,Audiberti,Jacques,petit,NORMANDIE


In [49]:
%%sql DROP TABLE IF EXISTS sales2;

-- Create sales2 table

CREATE TABLE sales2 AS
SELECT DISTINCT NB, NV, QTE, DATES, LIEU
FROM MASTER2
WHERE (NB IS NOT NULL) & (NV IS NOT NULL);

SELECT * FROM sales2;

 * sqlite:///wine.db
Done.
Done.
Done.


NB,NV,QTE,DATES,LIEU
2,1,33,1977-11-02,BORDEAUX
44,1,1,2015-10-16,PARIS
45,1,1,1983-12-31,RENNES
48,2,2,1983-12-25,LYON
7,3,6,1978-11-01,NICE
49,3,5,1987-12-24,MERCUREY
8,4,12,1982-11-05,TOULOUSE
44,4,6,2015-10-16,PARIS
50,4,4,1981-06-29,SENS
2,5,50,1984-11-04,MERCUREY


# PART II: SQL QUERIES

In the second part of this TP you will create SQL queries to retrieve information from the database.

**Important:**

- You MUST use the normalized tables created in previous steps.
  - The normalized tables will also be used in TP4.
- Do NOT use **Master1** and **Master2** in your queries.

#### Exercise 2.1

What are the different types of clients (buveurs) by volume of purchases?

In [50]:
%%sql
SELECT DISTINCT TYPE  as Type_of_buveurs FROM client;

 * sqlite:///wine.db
Done.


Type_of_buveurs
petit
moyen
gros


#### Exercise 2.2

What regions produce Pommard or Brouilly?

In [51]:
%%sql
SELECT DISTINCT CRU, REGION FROM wine1 w1, production p, sales1 s1
WHERE (s1.nv = w1.nv) AND (s1.np = p.np) AND (CRU = "Pommard" OR CRU = 'Brouilly');

 * sqlite:///wine.db
Done.


CRU,REGION
Pommard,Bourgogne
Pommard,Rhone
Brouilly,Bourgogne


#### Exercise 2.3

What regions produce Pommard and Brouilly?

In [52]:
%%sql
SELECT DISTINCT REGION FROM wine1 w1, production p, sales1 s1
WHERE (s1.nv = w1.nv) AND (s1.np = p.np) AND (CRU = "Pommard")
INTERSECT
SELECT DISTINCT REGION FROM wine1 w1, production p, sales1 s1
WHERE (s1.nv = w1.nv) AND (s1.np = p.np) AND (CRU = "Brouilly")

 * sqlite:///wine.db
Done.


REGION
Bourgogne


#### Exercise 2.4

Get the number of wines bught by CRU and Millésime

In [53]:
%%sql
SELECT DISTINCT CRU, MILL, SUM(QTE) as Number_of_wines_bought
FROM wine1 w2, sales2 s2
WHERE w2.nv = s2.nv
GROUP BY CRU, MILL
ORDER BY CRU, MILL DESC;

 * sqlite:///wine.db
Done.


CRU,MILL,Number_of_wines_bought
Arbois,1980,8
Auxey Duresses,1914,80
Beaujolais Primeur,1983,7
Beaujolais Villages,1979,520
Beaujolais Villages,1978,130
Beaujolais Villages,1976,120
Beaujolais Villages,1975,10
Chapelle Chambertin,1973,30
Chateau Corton Grancey,1980,4
Chenas,1984,1


#### Exercise 2.5

Retrieve the wine number (NV) of wines produced by more than three producers

In [54]:
%%sql
SELECT NV, count(NP)
FROM sales1
WHERE (NV IS NOT NULL) & (NP IS NOT NULL)
GROUP BY NV
HAVING count(NP) > 3;

 * sqlite:///wine.db
Done.


NV,count(NP)
45,5
78,5
89,4
98,5


#### Exercise 2.6

Which producers have not produced any wine?

In [55]:
%%sql
SELECT s1.NP, NOM, PRENOM
FROM production p, sales1 s1
WHERE (s1.np = p.np) & (s1.nv IS NULL);

 * sqlite:///wine.db
Done.


NP,NOM,PRENOM
3,Six,Paul
6,Marmagne,Bernard
8,Lioger d'Harduy,Gabriel
16,Barbin,Bernard
17,Faiveley,Guy
18,Tramier,Jean
19,Dupaquier,Roger
20,Lamy,Jean
21,Cornu,Edmond
26,Violot,Gilbert


#### Exercise 2.7

What clients (buveurs) have bought at least one wine from 1980?

In [56]:
%%sql
SELECT DISTINCT c.NB, c.NOM, c.PRENOM
FROM client c, wine2 w2, sales2 s2
WHERE (s2.nb = c.nb) & (s2.nv = w2.nv) &( MILL = '1980')
ORDER BY c.NB;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
2,Artaud,Antonin
8,Aragon,Louis
44,Gide,Andre
45,Giono,Jean
50,Lautreamont,None
61,Mallarme,Stephane


#### Exercise 2.8

What clients (buveurs) have NOT bought any wine from 1980?

In [57]:
%%sql
SELECT DISTINCT NB, NOM, PRENOM FROM CLIENT WHERE NB NOT IN
(SELECT DISTINCT c.NB
FROM client c, wine2 w2, sales2 s2
WHERE (s2.nb = c.nb) & (s2.nv = w2.nv) &( MILL = '1980')
ORDER BY c.NB)
ORDER BY NB;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
1,Aristote,None
3,Aron,Raymond
4,Apollinaire,Guillaume
5,Audiberti,Jacques
6,Arrabal,Fernando
7,Anouilh,Jean
9,Ajar,Emile
10,Andersen,Yann
11,Breton,Andre
12,Bataille,Georges


#### Exercise 2.9

What clients (buveurs) have bought ONLY wines from 1980?

In [122]:
%%sql
SELECT DISTINCT c.NB, c.NOM, C.PRENOM, w2.MILL
FROM client c, wine2 w2, sales2 s2
WHERE (s2.nb = c.nb) & (s2.nv = w2.nv)
ORDER BY c.NB
;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,MILL
1,Aristote,None,1975
2,Artaud,Antonin,1980
2,Artaud,Antonin,1976
2,Artaud,Antonin,1979
2,Artaud,Antonin,1980
2,Artaud,Antonin,1976
2,Artaud,Antonin,1979
3,Aron,Raymond,1981
3,Aron,Raymond,1979
3,Aron,Raymond,1978


In [96]:
%%sql
SELECT DISTINCT NB, NOM, PRENOM FROM CLIENT WHERE NB NOT IN(
SELECT DISTINCT c.NB
FROM client c, wine2 w2, sales2 s2
WHERE (s2.nb = c.nb) & (s2.nv = w2.nv) & (w2.MILL != 1980)
)
ORDER BY NB
;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
11,Breton,Andre
13,Barthes,Roland
16,Balzac,Honore de
18,Celine,Louis Ferdinand
20,Chateaubriand,Francois-Rene de
21,Corbiere,Tristan
23,Corneille,Pierre
25,Char,Rene
27,Dumas,Alexandre
29,Fournier,Alain


Done.


NB,NOM,PRENOM
44,Gide,Andre
45,Giono,Jean
50,Lautreamont,None


#### Exercise 2.10

List all wines from 1980

In [63]:
%%sql
SELECT NV, CRU, MILL, DEGRE
FROM wine1
WHERE MILL = 1980
UNION
SELECT NV, CRU, MILL, DEGRE
FROM wine2
WHERE MILL = 1980

 * sqlite:///wine.db
Done.


NV,CRU,MILL,DEGRE
1,Mercurey,1980,11.5
4,Mercurey,1980,10.9
16,Meursault,1980,12.1
20,Cote de Brouilly,1980,12.1
26,Chateau Corton Grancey,1980,None
28,Volnay,1980,11
43,Fleurie,1980,11.4
74,Arbois,1980,12
78,Etoile,1980,12
79,Seyssel,1980,11


#### Exercise 2.11

What are the wines from 1980 bought by NB=2?

In [68]:
%%sql
SELECT DISTINCT w.NV, w.CRU, w.MILL, w.DEGRE
FROM wine2 w, client c, sales2 s2
WHERE (c.nb = s2.nb) & (w.nv = s2.nv) & (w.MILL = 1980) & (c.nb = 2)

 * sqlite:///wine.db
Done.


NV,CRU,MILL,DEGRE
1,Mercurey,1980,11.5


#### Exercise 2.12

What clients (buveurs) have bought ALL the wines from 1980?

In [89]:
%%sql
SELECT NB, NOM, PRENOM FROM(
SELECT DISTINCT c.NB, c.NOM, c.PRENOM, count(w.NV) as cnv
FROM wine2 w, client c, sales2 s2
WHERE (c.nb = s2.nb) & (w.nv = s2.nv) & (w.MILL = 1980)
GROUP BY c.NB
ORDER BY count(w.NV) DESC)
WHERE cnv = 18;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
44,Gide,Andre
